# Bilan radiatif et transport méridien d'énergie

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

D'après Stull : https://geo.libretexts.org/Bookshelves/Meteorology_and_Climate_Science/Book%3A_Practical_Meteorology_(Stull)/11%3A_General_Circulation/11.02%3A_Section_3-

Le fichier de données au format netcdf (moyennes mensuelles CERES EBAF 4.1) doivent être téléchargé et placé dans le répertoire data : https://asdc.larc.nasa.gov/project/CERES/CERES_EBAF_Edition4.1

In [ ]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import gridspec

import numpy as np
import scipy
from scipy import integrate

import pandas as pd
from pandas import Series
from sklearn.linear_model import LinearRegression

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dir_data='./data/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

# Lecture des données

In [ ]:
file="CERES_EBAF_Edition4.1_200003-202111.nc"
data=xr.open_dataset(dir_data+file)
print(data)

In [ ]:
year1=input("année de départ : ")
year2=input("année de fin : ")

In [ ]:
data=data.sel(time=slice(year1,year2))
print(data)

# Bilan radiatif au sommet de l'atmosphère

In [ ]:
months=['January','February','March','April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December']
seasons = ['DJF','JJA','MAM','SON']

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[ lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
# Rayonnement solaire incident
solar_mean = lonflip(data['solar_mon'].mean('time')) # moyenne annuelle
solar_mean_z = solar_mean.mean(axis=1)  # moyenne annuelle et zonale
solar_mean_month = lonflip(data['solar_mon'].groupby('time.month').mean('time')) # moyenne mensuelle
solar_mean_month_z = solar_mean_month.mean(axis=2)  # moyenne mensuelle et zonale
solar_mean_season = lonflip(data['solar_mon'].groupby('time.season').mean('time')) # moyenne saisonnière
solar_mean_sz = solar_mean_season.mean(axis=2)  # moyenne saisonnière et zonale

# Rayonnement visible (solaire réfléchi)
toa_sw_mean = lonflip(data['toa_sw_all_mon'].mean('time'))  # moyenne annuelle
toa_sw_mean_z = toa_sw_mean.mean(axis=1) # moyenne annuelle et zonale
toa_sw_mean_month = lonflip(data['toa_sw_all_mon'].groupby('time.month').mean('time')) # moyenne mensuelle
toa_sw_mean_month_z = toa_sw_mean_month.mean(axis=2)  # moyenne mensuelle et zonale
toa_sw_mean_season = lonflip(data['toa_sw_all_mon'].groupby('time.season').mean('time')) # moyenne saisonnière
toa_sw_mean_sz = toa_sw_mean_season.mean(axis=2)  # moyenne saisonnière et zonale

# Rayonnement IR émis
toa_lw_mean = lonflip(data['toa_lw_all_mon'].mean('time'))  # moyenne annuelle
toa_lw_mean_z = toa_lw_mean.mean(axis=1) # moyenne annuelle et zonale
toa_lw_mean_month = lonflip(data['toa_lw_all_mon'].groupby('time.month').mean('time')) #  moyenne mensuelle
toa_lw_mean_month_z = toa_lw_mean_month.mean(axis=2) # moyenne mensuelle et zonale
toa_lw_mean_season = lonflip(data['toa_lw_all_mon'].groupby('time.season').mean('time'))  # moyenne saisonnière
toa_lw_mean_sz = toa_lw_mean_season.mean(axis=2) # moyenne saisonnière et zonale

# Royonnement net TOA
toa_net_mean = lonflip(data['toa_net_all_mon'].mean('time'))# moyenne annuelle
toa_net_mean_z = toa_net_mean.mean(axis=1) # moyenne annuelle et zonale
toa_net_mean_month = lonflip(data['toa_net_all_mon'].groupby('time.month').mean('time')) # moyenne mensuelle
toa_net_mean_month_z = toa_net_mean_month.mean(axis=2) # moyenne mensuelle et zonale
toa_net_mean_season = lonflip(data['toa_net_all_mon'].groupby('time.season').mean('time')) # moyenne saisonnière
toa_net_mean_sz = toa_net_mean_season.mean(axis=2) # moyenne saisonnière et zonale

lat=solar_mean.lat.values
lon=solar_mean.lon.values

# Bilan radiatif au sommet de l'atmosphère en moyenne zonale

In [ ]:
fig = plt.figure(figsize=(12,8))
fig.suptitle('Top Of Atmosphere Radiative Budget - zonal and annual mean', fontsize=16)
plt.title('CERES '+year1+'-'+year2, loc='center')

plt.xlabel('Latitude')
plt.ylabel('Radiation ($W/m^2$)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.plot(lat, solar_mean_z, label='Incoming Solar Radiation', color='red', linewidth=2)
plt.plot(lat, toa_sw_mean_z, label='Reflected Solar Radiation', color='purple', linewidth=2)
plt.plot(lat, solar_mean_z-toa_sw_mean_z, label='Absorbed SW Radiation', color='orange', linewidth=2)
plt.plot(lat, toa_lw_mean_z, label='Emitted LW Radiation', color='blue', linewidth=2)
plt.plot(lat, toa_net_mean_z, label='Net Radiation', color='black', linewidth=2)
plt.axvline(0, color='grey', linestyle="-", linewidth=0.5)
plt.axhline(0, color='grey', linestyle="-", linewidth=0.5)
plt.fill_between(lat, toa_net_mean_z, where=toa_net_mean_z > 0, color='lightcoral')
plt.fill_between(lat, toa_net_mean_z, where=toa_net_mean_z < 0, color='lightcyan')
plt.legend(loc='lower center', ncol=3)

figname=dir_figs+'TOA_rad_budget_zonal_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Approximation analytique du bilan radiatif

<div class="alert alert-danger">
<p>On approxime les variations méridiennes de la température par la fonction $T_m(lat) = a + b [cos^3(lat)(1+\frac{3}{2}*sin^2(lat)]$</p>
    
<p>$a=a_1-\gamma*z$</p>
<p>$b=b_1*(1-\frac{z}{z_T})$</p>
<br>

<p>$z$ : altitude</p>
<p>$a_1 = –12 °C$</p>
<p>$\gamma = 3,14 °C.km^{–1}$</p>
<p>$z_T = 11 km$ : altitude de la tropopause</p>
<p>$b_1 = 40°C$ : différence de température équateur-pôle près de la surface</p>

<br>
<b>Tracer cette fonction pour les altitudes $z = 0 km$, $z = 1,5 km$, $z = 5,5 km$, $z = 10 km$ et $z = 15 km$</b>

</div>

In [ ]:
zt=11 # km, average tropospheric depth
b1=40 # °C, temperature difference between equator and pole
a1=-12 # °C
gamma=3.14 # °C/km

def T(z):
    b=b1*(1-(z/zt))
    a=a1-gamma*z
    return a+b*(np.cos(lat*np.pi/180)**3*(1+1.5*np.sin(lat*np.pi/180)**2))

fig = plt.figure(figsize=(15,8))

plt.xlabel('Latitude')
plt.ylabel('Temperature (K)')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))
plt.title('Idealized variation of annual-average temperature')
plt.axvline(0, color='black', linestyle="--")

for alt in [0,1.5,5.5,10,15]:
    plt.plot(lat, T(alt), label=str(alt)+'km')
plt.legend()
figname='./figs/ideal_temp'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-danger">
<p>On approxime les variations méridiennes du rayonnement incident par :</p>
<p>$E_{sol}=E_0+E_1*cos(2*lat)$</p>
<p>$E_0 = 298 W/m^2, E1 = 123 W/m^2$</p>
<br>

<p>On considère le rayonnement réfléchi constant avec la latitude :</p>
<p>$E_2 = 110 W/m^2$</p>
<p>Le rayonnement absorbé est donc :</p>
<p>$E_{in} = E_{sol} - E_2$</p>
<br>

<p> On suppose que l'émission infrarouge est caractéristique d'une température de moyenne troposphère $T_m$ (en Kelvin) à $z_m = 5,5 km$ et on approxime cette émission par la loi de Stefan-Boltzmann :</p>

<p>$E_{out}=\epsilon \sigma T^4$</p>

<p>$\epsilon=0,9$</p>
<p>$\sigma=5,67*10^{-8}Wm^{-2}K^{-4}$</p>
<br>
    
<b>Tracer les courbes théoriques de $E_{sol}$, $E_{out}$, $E_{in}$ et le bilan net $E_{in}-E_{out}$. Superposer ces courbes aux courbes des observations CERES. L'approximation analytique est-elle acceptable ?</b>

</div>

In [ ]:
# Approximation of the Meridional variation of insolation
E0=298 # W/m2
E1=123 # W/m2
Esol=E0+E1*np.cos(2*lat*np.pi/180)
E2=110 # W/m2, réfléchi
Ein=Esol-E2

# Approximation of the Outgoing longwave radiation
epsilon=0.9
sigma=5.67e-8
Tm=T(5.5)+273.15
Eout=epsilon*sigma*Tm**4

# Approximation of the net radiation
Enet=Ein-Eout

fig = plt.figure(figsize=(12,8))
fig.suptitle('Top Of Atmosphere Radiative Budget - zonal and annual mean', fontsize=16)
plt.title('CERES '+year1+'-'+year2+' VS toy model', loc='center')

plt.xlabel('Latitude')
plt.ylabel('Radiation ($W/m^2$)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))

plt.plot(lat, solar_mean_z, label='Incoming Solar Radiation', color='red', linewidth=2)
plt.plot(lat, Esol, label='$E_{sol}=E_0+E_1*cos(2𝜙)$', color='red', linestyle='--', linewidth=2)

plt.plot(lat, solar_mean_z-toa_sw_mean_z, label='Absorbed SW Radiation', color='orange', linewidth=2)
plt.plot(lat, Ein, label='$E_{in}=E_{sol}-110$', color='orange', linestyle='--', linewidth=2)

plt.plot(lat, toa_lw_mean_z, label='Emitted LW Radiation', color='blue', linewidth=2)
plt.plot(lat, Eout, label='$E_{out}=\sigma T_m^4$', color='blue', linestyle='--', linewidth=2)

plt.plot(lat, toa_net_mean_z, label='Net Radiation', color='black', linewidth=2)
plt.plot(lat, Enet, label='$E_{in}-E_{out}$', color='black', linestyle='--', linewidth=2)

plt.axvline(0, color='grey', linestyle="-", linewidth=0.5)
plt.axhline(0, color='grey', linestyle="-", linewidth=0.5)
plt.fill_between(lat, toa_net_mean_z, where=toa_net_mean_z > 0, color='lightcoral')
plt.fill_between(lat, toa_net_mean_z, where=toa_net_mean_z < 0, color='lightcyan')
plt.legend(loc='lower center', ncol=4)

figname=dir_figs+'TOA_rad_budget_zonal_annual_analytic'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Chauffage différentiel et transport méridien d'énergie

<div class="alert alert-danger">
<p>La circonférence d'un parallèle (cercle de latitude constante) est plus petite près des pôles qu'à l'équateur.</p>

<br>
    
<b>Multiplier le bilan radiatif net algébrique par la circonférence d'un parallèle pour convertir le rayonnement (unité $W/m^2$) en $E_ϕ$ (unité $W/m$). $E_ϕ$ peut être interprété comme la puissance transférée vers (où depuis) une section de 1m qui entoure la Terre le long d'un parallèle.</b>

</div>

In [ ]:
re=6371000
mult=2*np.pi*re*np.cos(lat*np.pi/180)

fig = plt.figure(figsize=(12,8))
fig.suptitle('Top Of Atmosphere net Radiation - zonal and annual mean', fontsize=16)
plt.title('Toy model')

plt.xlabel('Latitude')
plt.ylabel('$E_\phi (GW/m)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.plot(lat, Enet*mult*1e-9, label='Net', color='black', linewidth=2)
plt.fill_between(lat, Enet*mult*1e-9, where=Enet*mult*1e-9 > 0, color='lightcoral')
plt.fill_between(lat, Enet*mult*1e-9, where=Enet*mult*1e-9 < 0, color='lightcyan')
plt.axvline(0, color='black', linestyle="-", linewidth=0.5)
plt.axhline(0, color='black', linestyle="-", linewidth=0.5)

figname=dir_figs+'TOA_net_area_analytic'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-danger">
<p>La quantité $E_ϕ$ peut être définie comme un chauffage différentiel. Le transport méridien à chaque pôle est nul car le pôle est un point où convergent les méridiens. En sommant $E_ϕ$ pour toutes les ceintures de latitudes du pôle nord à n'importe quelle latitude on peut calculer le transport méridien total nécessaire pour que la circulation globale compense tous les déséquilibres méridiens au nord de cette latitude </p>

<br>
    
<b>Utiliser la fonction scipy.integrate.cumtrapz pour implémenter le calcul du transport $Tr⁡(lat)=-\sum_{lat=90°}^{-90°} E_ϕ \Delta y$ et tracer sa distribution méridienne.</b>
    
<br>

<b>La largeur de la ceinture de latitude $\Delta y$ est relée à la variation de latitude $\Delta lat$ par : $\Delta y(km) = (111 km/°)*\Delta lat (°)$</b>

</div>

In [ ]:
print(lat[::-1])

import scipy
from scipy import integrate

dphi=lat[1]-lat[0]
Tphi=-Enet*mult*111000*dphi
T=scipy.integrate.cumtrapz(Tphi[::-1], x=lat[::-1], axis=0, initial=0)

fig = plt.figure(figsize=(12,8))
fig.suptitle('Required heat transport to compensate radiative differential heating', fontsize=16)
plt.title('Toy model')
plt.xlabel('Latitude')
plt.ylabel('Transport (PW)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.plot(lat, T*1e-15, label='Transport', color='black', linewidth=2)
plt.axvline(0, color='black', linestyle="-", linewidth=0.5)
plt.axhline(0, color='black', linestyle="-", linewidth=0.5)

figname=dir_figs+'TOA_net_transport_analytic'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-danger">
<p>Comparer avec le transport méridien réel ci-dessous :
    K. E. Trenberth and J. M. Caron, 2001: “J. Climate”, 14, 3433-3443.</p>

</div>

<img src="tr.png" width="600">